In [21]:
import awswrangler as wr
import pandas as pd
import datetime
import numpy as np

rutaCCHC = f"sap/cchc/pedidos_historicos/extraction_year=2023/"
rutaSSLC = f"sap/sslc/pedidos_historicos/extraction_year=2023/"

staging_bucket = 'cchc-dw-qa-staging'
raw_bucket = 'cchc-dw-qa-raw'

dfSSLC = wr.s3.read_parquet(path=f"s3://{raw_bucket}/{rutaSSLC}")
dfCCHC = wr.s3.read_parquet(path=f"s3://{raw_bucket}/{rutaCCHC}")

SSLC

In [ ]:
def cast_to_pydate(fecha:str):
    if fecha is None or str(fecha) == '<NA>':
        return None

    return datetime.datetime.strptime(fecha, "%Y-%m-%d %H:%M:%S").date()

In [ ]:
df = dfSSLC.copy()

new_cols = ['sucursal_pedido', 'ceco_pedido', 'nombre_ceco_pedido', 'proyecto_pedido', 'estatus_pedido', 'desc_estatus_pedido',
            'fecha_cont_pedido', 'fecha_creacion_pedido', 'nro_pedido', 'nro_linea_pedido', 'codigo_pedido', 'descrpcion_pedido', 'texto_pedido',
            'cod_proveedor', 'nombre_proveedor', 'moneda', 'cantidad_pedido', 'precio_unitario_pedido', 'total_linea_pedido', 'cod_impuesto_pedido',
            'cant_pendiente_pedido', 'tasa_de_cambio_pedido', 'importe_pendiente_pedido', 'cta_contable_pedido', 'nombre_cta_contable_pedido',
            'nro_solicitud', 'usuario_solped', 'fecha_creacion_solped', 'nro_entrada', 'nro_interno_entrada', 'estatus_em', 'fecha_cont_em', 'fecha_creacion_em',
            'fecha_venc_em', 'texto_em', 'comentario_em','ceco_em', 'nombre_ceco_em', 'cantidad_em', 'precio_unitario_em', 'total_linea_em_clp', 'cod_impuesto_em',
            'impuesto', 'tasa_de_cambio_em', 'usuario_em', 'nro_interno_devolucion', 'nro_devolucion', 'nro_linea_dev', 'nro_intsoldev', 'nro_soldev',
            'nro_linea_soldev', 'nro_interno_factura', 'nro_factura', 'nro_folio_factura', 'fecha_cont_factura', 'fecha_creacion_factura',
            'fecha_venc_factura', 'nro_linea_fact', 'monto_pago_fact', 'doc_base', 'nro_sap_ag', 'estatus_ag_cancelado', 'fecha_inicio_ag', 'fecha_fin_ag',
            'tipo_asignacion', 'continuidad_servicio', 'texto_libre_ag', 'ref_iconstruye', 'monto_disponible_pedidos_oc', 'monto_ingresos_efectuado_em',
            'monto_disponible_ag_clp', 'moneda_del_ag', 'fecha_inicio_contrato', 'fecha_fin_contrato', 'propietario', 'fecha_extraccion']


df.columns = new_cols


int32types = ["nro_pedido", "nro_linea_pedido", "nro_solicitud", "nro_entrada", "nro_interno_entrada", "nro_interno_devolucion", "nro_devolucion",
     "nro_linea_dev", "nro_intsoldev", "nro_intsoldev", "nro_linea_soldev", "nro_interno_factura", "nro_factura", "nro_folio_factura", "nro_sap_ag"]

for campo in int32types:
    df[campo] = df[campo].astype("Int32")

floats = ["cant_pendiente_pedido", "tasa_de_cambio_pedido",
            "cantidad_pedido"]
for campo in floats:
    df[campo] = df[campo].astype("float")

doubles = ["precio_unitario_pedido", "importe_pendiente_pedido", "monto_disponible_pedidos_oc", "monto_ingresos_efectuado_em",
            "monto_disponible_ag_clp"]
for campo in doubles:
    df[campo] = df[campo].astype("double")

nullable_ints = ["total_linea_em_clp", "impuesto"]
for campo in nullable_ints:
    print(campo)
    df[campo] = df[campo].astype('Int64')

strings = ["ceco_pedido", "nombre_ceco_pedido", "proyecto_pedido", "texto_pedido", "descrpcion_pedido",
            "texto_pedido", "cod_proveedor", "nombre_proveedor", "moneda", "cod_impuesto_pedido", "nombre_cta_contable_pedido",
            "usuario_solped", "estatus_em", "texto_em", "ceco_em", "nombre_ceco_em", "cantidad_em", "cod_impuesto_em",
            "tasa_de_cambio_em", "usuario_em", "estatus_em", "estatus_ag_cancelado", "continuidad_servicio", "texto_libre_ag",
            "ref_iconstruye", "moneda_del_ag", "propietario", "tipo_asignacion"]
for col in strings:
    df.loc[(df[col].isnull()) | (df[col] == 'nan'), col] = None
    df[col] = df[col].astype("string")

fechas = ["fecha_cont_pedido", "fecha_creacion_pedido", "fecha_cont_em", "fecha_creacion_em", "fecha_creacion_solped", "fecha_venc_em", "fecha_creacion_factura",
            "fecha_venc_factura", "fecha_inicio_ag", "fecha_fin_ag", "fecha_inicio_contrato", "fecha_cont_factura", "fecha_venc_factura"]

df["fecha_fin_contrato"] = df.fecha_fin_contrato.apply(lambda x: cast_to_pydate(x))

for campo in fechas:
    print(campo)
    df[campo] = df[campo].astype("datetime64")

df["cod_impuesto_em"] = df["cod_impuesto_em"].fillna("_")
df["monto_total"] = np.where(df["cod_impuesto_em"] == "IVA_EXE",
                                df["total_linea_em_clp"], df["total_linea_em_clp"] * 1.19)

df["dsc_estatus_pedido"] = np.where(
    df["estatus_pedido"] == "O", 'Pedido Abierto', 'Pedido Cerrado')

# Unir dataframe con el listado de proyectos para agregar el nombre

proyectosDf = wr.s3.read_parquet(
    path=f"s3://{raw_bucket}/sap/sslc/oprj/extraction_year=2023/")
proyectosDf.columns = ["proyecto_pedido", "proyecto", "extraction_date"]

proyectosDf = proyectosDf.drop(columns=["extraction_date"])

_MESES = [
    'Enero',
    'Febrero',
    'Marzo',
    'Abril',
    'Mayo',
    'Junio',
    'Julio',
    'Agosto',
    'Septiembre',
    'Octubre',
    'Noviembre',
    'Diciembre'
]

for i, m in enumerate(_MESES):
    _MESES[i] = {
        "fecha_cont_em_month": i + 1,
        "fecha_cont_em_mes_spa": m
    }

df["fecha_cont_em_year"] = df["fecha_cont_em"].dt.year
df["fecha_cont_em_month"] = df["fecha_cont_em"].dt.month

meses_df = pd.DataFrame(data=_MESES)
df = pd.merge(df, meses_df, on=["fecha_cont_em_month"], how="left")

df["fecha_cont_em_year"] = df["fecha_cont_em_year"].astype("Int32")
df["fecha_cont_em_month"] = df["fecha_cont_em_month"].astype("Int32")

print(df)

for campo in fechas:
    df[campo] = pd.to_datetime(df[campo], errors="coerce").dt.date

df = pd.merge(df, proyectosDf, on=["proyecto_pedido"], how="left")

ruta = f"s3://{staging_bucket}/sap/sslc/pedidos_historicos/"

wr_r = wr.s3.to_parquet(df=df,
                    path=ruta,
                    dataset=True,
                    mode="overwrite",
                    table="sslc_pedidos_historicos",
                    database='staging_qa',
                    schema_evolution=True)

CCHC

In [ ]:
df = dfCCHC.copy()

new_cols = ['sucursal_pedido', 'ceco_pedido', 'nombre_ceco_pedido', 'proyecto_pedido', 'estatus_pedido', 'desc_estatus_pedido',
            'fecha_cont_pedido', 'fecha_creacion_pedido', 'nro_pedido', 'nro_linea_pedido', 'codigo_pedido', 'descrpcion_pedido', 'texto_pedido',
            'cod_proveedor', 'nombre_proveedor', 'moneda', 'cantidad_pedido', 'precio_unitario_pedido', 'total_linea_pedido', 'cod_impuesto_pedido',
            'cant_pendiente_pedido', 'tasa_de_cambio_pedido', 'importe_pendiente_pedido', 'cta_contable_pedido', 'nombre_cta_contable_pedido',
            'nro_solicitud', 'usuario_solped', 'fecha_creacion_solped', 'nro_entrada', 'nro_interno_entrada', 'estatus_em', 'fecha_cont_em', 'fecha_creacion_em',
            'fecha_venc_em', 'texto_em', 'comentario_em', 'ceco_em', 'nombre_ceco_em', 'cantidad_em', 'precio_unitario_em', 'total_linea_em_clp', 'cod_impuesto_em',
            'impuesto', 'tasa_de_cambio_em', 'usuario_em', 'nro_interno_devolucion', 'nro_devolucion', 'nro_linea_dev', 'nro_intsoldev', 'nro_soldev',
            'nro_linea_soldev', 'nro_interno_factura', 'nro_factura', 'nro_folio_factura', 'fecha_cont_factura', 'fecha_creacion_factura',
            'fecha_venc_factura', 'nro_linea_fact', 'monto_pago_fact', 'doc_base', 'nro_sap_ag', 'estatus_ag_cancelado', 'fecha_inicio_ag', 'fecha_fin_ag',
            'tipo_asignacion', 'continuidad_servicio', 'texto_libre_ag', 'ref_iconstruye', 'monto_disponible_pedidos_oc', 'monto_ingresos_efectuado_em',
            'monto_disponible_ag_clp', 'moneda_del_ag', 'fecha_inicio_contrato', 'fecha_fin_contrato', 'propietario', 'fecha_extraccion']

df.columns = new_cols

int32types = ["nro_pedido", "nro_linea_pedido", "nro_solicitud", "nro_entrada", "nro_interno_entrada", "nro_interno_devolucion", "nro_devolucion",
    "nro_linea_dev", "nro_intsoldev", "nro_intsoldev", "nro_linea_soldev", "nro_interno_factura", "nro_factura", "nro_folio_factura", "nro_sap_ag"]
for campo in int32types:
    df[campo] = df[campo].astype("Int32")

floats = ["cant_pendiente_pedido", "tasa_de_cambio_pedido",
            "cantidad_pedido"]
for campo in floats:
    df[campo] = df[campo].astype("float")

doubles = ["precio_unitario_pedido", "importe_pendiente_pedido", "monto_disponible_pedidos_oc", "monto_ingresos_efectuado_em",
            "monto_disponible_ag_clp"]
for campo in doubles:
    df[campo] = df[campo].astype("double")

nullable_ints = ["total_linea_em_clp", "impuesto"]
for campo in nullable_ints:
    print(campo)
    df[campo] = df[campo].astype('Int64')

strings = ["ceco_pedido", "nombre_ceco_pedido", "proyecto_pedido", "texto_pedido", "descrpcion_pedido",
            "texto_pedido", "cod_proveedor", "nombre_proveedor", "moneda", "cod_impuesto_pedido", "nombre_cta_contable_pedido",
            "usuario_solped", "estatus_em", "texto_em", "ceco_em", "nombre_ceco_em", "cantidad_em", "cod_impuesto_em",
            "tasa_de_cambio_em", "usuario_em", "estatus_em", "estatus_ag_cancelado", "continuidad_servicio", "texto_libre_ag",
            "ref_iconstruye", "moneda_del_ag", "propietario", "tipo_asignacion"]
for col in strings:
    df.loc[(df[col].isnull()) | (df[col] == 'nan'), col] = None
    df[col] = df[col].astype("string")


df["fecha_fin_contrato"] = df.fecha_fin_contrato.apply(lambda x: cast_to_pydate(x))

fechas = ["fecha_cont_pedido", "fecha_creacion_pedido", "fecha_cont_em", "fecha_creacion_em", "fecha_venc_em", "fecha_creacion_solped", "fecha_creacion_factura",
            "fecha_venc_factura", "fecha_inicio_ag", "fecha_fin_ag", "fecha_inicio_contrato", "fecha_cont_factura", "fecha_venc_factura"]

for campo in fechas:
    print(campo)
    df[campo] = df[campo].astype("datetime64")

df["cod_impuesto_em"] = df["cod_impuesto_em"].fillna("_")
df["monto_total"] = np.where(df["cod_impuesto_em"] == "IVA_EXE",
                                df["total_linea_em_clp"], df["total_linea_em_clp"] * 1.19)

df["dsc_estatus_pedido"] = np.where(
    df["estatus_pedido"] == "O", 'Pedido Abierto', 'Pedido Cerrado')

# Unir dataframe con el listado de proyectos para agregar el nombre

proyectosDf = wr.s3.read_parquet(
    path=f"s3://{raw_bucket}/sap/cchc/oprj/extraction_year=2023/")
proyectosDf.columns = ["proyecto_pedido", "proyecto", "extraction_date"]

proyectosDf = proyectosDf.drop(columns=["extraction_date"])

_MESES = [
    'Enero',
    'Febrero',
    'Marzo',
    'Abril',
    'Mayo',
    'Junio',
    'Julio',
    'Agosto',
    'Septiembre',
    'Octubre',
    'Noviembre',
    'Diciembre'
]

for i, m in enumerate(_MESES):
    _MESES[i] = {
        "fecha_cont_em_month": i + 1,
        "fecha_cont_em_mes_spa": m
    }

df["fecha_cont_em_year"] = df["fecha_cont_em"].dt.year
df["fecha_cont_em_month"] = df["fecha_cont_em"].dt.month

meses_df = pd.DataFrame(data=_MESES)
df = pd.merge(df, meses_df, on=["fecha_cont_em_month"], how="left")

df["fecha_cont_em_year"] = df["fecha_cont_em_year"].astype("Int32")
df["fecha_cont_em_month"] = df["fecha_cont_em_month"].astype("Int32")

print(df)

for campo in fechas:
    df[campo] = pd.to_datetime(df[campo], errors="coerce").dt.date

df = pd.merge(df, proyectosDf, on=["proyecto_pedido"], how="left")

ruta = f"s3://{staging_bucket}/sap/cchc/pedidos_historicos/"

wr.s3.to_parquet(
    df=df,
    path=ruta,
    dataset=True,
    mode="overwrite",
    table="cchc_pedidos_historicos",
    database='staging_qa',
    compression="gzip"
    )